# Olympic Results and Country GDP and Popluation Data 

## Overview

#### This notebook contains the python code to populate a database that contains tables: 1. olympic_data; 2. athlete_data; 3. country_data.  The data for these tables comes from csv files containing olympic results from 120 years of Olympic Games and world population and GDP data.

* Olypmic and Athlete data comes from two csv files (athlete_events.csv and noc_regions.csv)found at: https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?resource=download

* World Population data comes from one csv file (Population.csv) found at: https://data.worldbank.org/indicator/SP.POP.TOTL

* World GDP data comes from one csv file (GDP.csv) found at: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2021&start=1960

#### *Please note that you need to have created the database and tables prior to running this notebook to load the data

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import numpy as np

## Extract

#### Each of the 4 csv files (athlete_events.csv, noc_regions.csv, Population.csv, GDP.csv) in the Resources folder is extracted into a dataframe using pd.read_csv


In [2]:
#Extract athlete_events.csv into olympic_df dataframe
olympic_file = "Resources/athlete_events.csv"
olympic_df = pd.read_csv(olympic_file)

In [3]:
#Extract noc_regions.csv into country_df dataframe
country_file = "Resources/noc_regions.csv"
country_df = pd.read_csv(country_file)

In [4]:
#Extract Population.csv into pop_df dataframe
pop_file = "Resources/Population.csv"
pop_df = pd.read_csv(pop_file, skiprows=4)

In [5]:
#Extract GDP.csv into gdp_df dataframe
gdp_file = "Resources/GDP.csv"
gdp_df = pd.read_csv(gdp_file, skiprows=4)

## Transform

#### Transform each of the 4 dataframes to reduce columns, replace null values, merge data and ultimately get it into the format needed for loading to the database.

### 1. Transform Olympic DataFrame to athlete data for athlete_data table
* Reduce columns to ID, Sex, Age
* Rename columns
    * ID to athlete_id
    * All changed to lower case

In [6]:
# Create a filtered dataframe from specific columns for the Athlete_Data Table
athlete_cols = ["ID", "Sex", "Age"]
athlete_transformed= olympic_df[athlete_cols].copy()

# Rename the column headers
athlete_transformed = athlete_transformed.rename(columns={"ID": "athlete_id",
                                                          "Sex": "sex",
                                                          "Age": "age"})

# Clean the data by dropping duplicates and setting the index
athlete_transformed.drop_duplicates("athlete_id", inplace=True)
athlete_transformed.fillna(0, inplace=True)


### 2. Transform: Olympic DataFrame and Country DataFrame to load to olympic_data table

##### Olympic Dataframe
* Reduce columns to ID, Year, Season, Sport, Event, Medal, NOC
* Rename columns 
    * ID to athlete_id
    * All changed to lower case (other than NOC)
* Remove all data with year prior to 1960
* Replace 'NaN' to 'none' in columns

##### Country Dataframe
* Rename columns 
    * region to country
* Replace 'NaN' to 'none' in columns
* Update some Countries to match the world data
    * USA = United States
    * GBR = United Kingdom

##### Merge the olympic and country dataframes
* Use NOC' to merge the dataframes. Now the merged dataframe will have the country field
* Drop the 'NOC' field
* Replace 'NaN' with '0'

#### The merged dataframe is what will be loaded to olympic_data table

In [7]:
##### Transform Olympic DataFrame #####

# Create a filtered dataframe from specific columns for the Olympic_Data Table
olympic_cols = ["ID", "Year", "Season", "Sport", "Event", "Medal", "NOC"]
olympic_transformed= olympic_df[olympic_cols].copy()

# Rename the column headers
olympic_transformed = olympic_transformed.rename(columns={"ID": "athlete_id",
                                                          "Year": "year",
                                                          "Season": "season",
                                                          "Sport": "sport",
                                                          "Event": "event",
                                                          "Medal": "medal"})

#Replace medal NaN to none
olympic_transformed.fillna('None', inplace=True)

##### Transform country DataFrame #####

# Create a filtered dataframe from specific columns needed to add the country to olympic_data Table
country_cols = ["NOC", "region"]
country_transformed = country_df[country_cols].copy()

# Rename the column headers
country_transformed = country_transformed.rename(columns={"region": "country"})
country_transformed.head()

# Update USA and GBR to correct names
country_transformed.loc[country_transformed["country"] == "USA", "country"] = "United States"
country_transformed.loc[country_transformed["country"] == "UK", "country"] = "United Kingdom"

##### Replace NOC in Olympic Dataset with Country #####
olympic_m_df = olympic_transformed.merge(country_transformed, on='NOC')
olympic_m_df.drop('NOC', axis=1, inplace=True)

#Replace country NaN to unkown
olympic_m_df.fillna('unkown', inplace=True)
len(olympic_m_df)

#Remove data prior to 1960
olympic_clean = olympic_m_df[olympic_m_df.year > 1959]


### 3. Transform: Country Population and Country GDP

##### Population Dataframe
* Drop columns Country Code, Indicator Name, Indicator Code
* Rename columns 
    * Country Name to country
* Melt the dataframe so that it will have a new row for each of the 
    * Before the melt the datafram had columns country and one for each year of data (1960, 1960, etc)
    * After the melt each year's data is stored in a new column for each country and rows with columns: country, year, pop(value)
* Rename column 'value' to 'population'
* Set null values to 0

##### GDP Dataframe
* Drop columns Country Code, Indicator Name, Indicator Code
* Rename columns 
    * Country Name to country
* Melt the dataframe so that it will have a new row for each of the 
    * Before the melt the datafram had columns country and one for each year of data (1960, 1960, etc)
    * After the melt each year's data is stored in a new column for each country and rows with columns: country, year, gdp(value)
* Rename column 'value' to 'gdp'
* Set null values to 0

##### Merge GDP and Population
* Merge GDP and Population datasets on country and year to create one dataset with fields
    * country
    * year
    * population
    * gdp

In [8]:
##### Country Population #####

# eliminate unused columns
pop_df.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)

# rename columns to what will be used in the database
pop_df.rename(columns={'Country Name' : 'country'}, inplace=True)

# get all the column names
cols = pop_df.columns.values.tolist()

# drop the country column name
cols.pop(0)

# convert all the year columns into rows
pop_m_df = pop_df.melt(id_vars = 'country', value_vars = cols, var_name = 'year')

#rename value to population
pop_m_df.rename(columns={'value' : 'population'}, inplace=True)

# set null values to 0
cleaned_pop_df = pop_m_df.fillna(0)



In [9]:
##### Country GDP #####

# eliminate unused columns
gdp_df.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)

# rename columns to what will be used in the database
gdp_df.rename(columns={'Country Name' : 'country'}, inplace=True)

# get all the column names
cols = gdp_df.columns.values.tolist()

# drop the country column name
cols.pop(0)

# convert all the year columns into rows
gdp_m_df = gdp_df.melt(id_vars = 'country', value_vars = cols, var_name = 'year')

# rename value to gdp
gdp_m_df.rename(columns={'value' : 'gdp'}, inplace=True)

# set null values to 0
cleaned_gdp_df = gdp_m_df.fillna(0)



In [10]:
##### Merge Population and GDP Dataframes #####

# merge dataframes keeping only rows that have the same country and year and adding the remaining columns
merged_country_df = pd.merge(cleaned_gdp_df,cleaned_pop_df, how="inner", on=['country','year'])

## Load 
##### Load the 3 DataFrames containing athlete data, olympic data and country data into the postgres database using pandas 'to_sql' command.

##### *Please note:  You may need to change the username, password and port data in the cell below in order to connect to your PostgreSQL database.

In [11]:
##### Create Database Connection #####

protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'olympic_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

inspector = inspect(engine)
inspector.get_table_names()

['country_data', 'athlete_data', 'olympic_data']

In [12]:
#Upload to athlete_data
athlete_transformed.to_sql(name='athlete_data', con=engine, if_exists='append', index=False)

571

In [13]:
#Upload to country_data
merged_country_df.to_sql(name='country_data', con=engine, if_exists='append', index=False)

492

In [14]:
#Upload to olympic_data
olympic_clean.to_sql(name='olympic_data', con=engine, if_exists='append', index=False)

783